# 阿里云DSW 一键脚本 By bilibili@
## 参考感谢——秋葉aaaki，十字鱼

基础环境：stable-diffusion-webui-develop:1.0-pytorch2.0-gpu-py310-cu117-ubuntu22.04
- Python:3.10.12
- CUDA: 11.7
- Pytorch: 2.0.1+cu117
- 显卡: NVIDIA A10


## 常用工具

In [ ]:
# 检查显卡
import torch
print('CUDA版本:',torch.version.cuda)
print('Pytorch版本:',torch.__version__)
print('显卡是否可用:','可用' if(torch.cuda.is_available()) else '不可用')
print('显卡数量:',torch.cuda.device_count())
print('是否支持BF16数字格式:','支持' if (torch.cuda.is_bf16_supported()) else '不支持不能训练sdxl')
print('当前显卡型号:',torch.cuda.get_device_name())
print('当前显卡的CUDA算力:',torch.cuda.get_device_capability())
print('当前显卡的总显存:',torch.cuda.get_device_properties(0).total_memory/1024/1024/1024,'GB')
print('是否支持TensorCore:','支持' if (torch.cuda.get_device_properties(0).major >= 7) else '不支持')
print('当前显卡的显存使用率:',torch.cuda.memory_allocated(0)/torch.cuda.get_device_properties(0).total_memory*100,'%')

In [ ]:
# 查看剩余内存空间
import subprocess
# 执行命令并解析输出
command = "df -h /mnt/workspace"
result = subprocess.check_output(command, shell=True, text=True)
data = result.split('\n')[1].split()
# 提取空间信息
文件系统, 总共, 已使用, 剩余 = data[0], data[1], data[2], float(data[3][:-1])
# 定义ANSI转义码
BOLD, RED, RESET = "\033[1m", "\033[31m", "\033[0m"
# 输出信息
print(f"{BOLD}文件系统: {文件系统}{RESET}")
print(f"{BOLD}总共空间: {总共}{RESET}")
print(f"{BOLD}已使用空间: {已使用}{RESET}")
# 检查剩余空间并输出相应信息
if 剩余 < 20:
    print(f"{BOLD}{RED}磁盘空间不足20GB，请及时处理！{RESET}")
else:
    print(f"磁盘空间充足，剩余空间为 {剩余} GB")

In [ ]:
# 3.压缩输出并清空
%cd /mnt/workspace/ComfyUI
!zip -r output.zip output
!mv output.zip ..
!rm -rf output

## 一、 安装Lora-Scipts
- 如果启动不成功，请停止后重复运行第1步。
- 使用【秋葉aaaki】脚本库
- 安装默认为最新版

In [1]:
import os
import time
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
from huggingface_hub import HfApi, list_repo_files
#安装目录
path = "/mnt/workspace"
#安装网址
url2 = "https://github.com/Akegarasu/lora-scripts"
repo2 = url2.split('/')[-1]
#修改分支
branch2 = "main"
#SD模型（先换行，再加链接）
model_urls = """
"""
#unet模型（先换行，再加链接）
unet_urls = """
"""
#VAE模型（先换行，再加链接）
VAE_urls = """
"""
#lora模型（先换行，再加链接）
lora_urls = """
"""
#ControlNet模型（先换行，再加链接）
ControlNet_urls = """
"""
#扩展插件（先换行，再加链接,不要带.git）
extension_urls = """
"""
#embedding模型（先换行，再加链接）
embedding_urls = """
"""
#运动lora（先换行，再加链接）
motion_lora_urls = """
"""
#视频文件（先换行，再加链接）
video_urls = """
"""
#额外文件（先换行，再加链接）（下载到安装目录，方便剪切）
extra_urls = """
"""
#内存优化（不要修改）
def libtcmalloc():
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -qq -y zip > /dev/null 2>&1
    !apt-get install -qq -y ffmpeg > /dev/null 2>&1
    !apt-get install -qq -y gifsicle > /dev/null 2>&1
    !apt-get install -qq -y libimage-exiftool-perl > /dev/null 2>&1
    !DEBIAN_FRONTEND="noninteractive" apt-get install -y python3-tk > /dev/null 2>&1
    if os.path.exists(f'{path}/temp'):
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm -rf /mnt/workspace/temp
        print('内存优化已安装1')
    else:
        os.chdir(f'{path}')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt-get install -qq libunwind8-dev -y > /dev/null 2>&1')
        !dpkg -i *.deb > /dev/null 2>&1
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb > /dev/null 2>&1
        !rm -rf /mnt/workspace/temp
        print('内存优化安装完毕2')
#安装主体（不要修改）
def install_main():
    print(f'开始安装{repo2}')
    if os.path.exists(f'{path}/{repo2}'):
        %cd {path}/{repo2}
        print(f'开始安装{repo2}依赖，请耐心等待')
        !pip install -r requirements.txt > /dev/null 2>&1
        !pip install gradio-tunneling > /dev/null 2>&1
        print(f'{repo2}升级完毕')
#下载文件1（不要修改）
def download_1():
    tasks = []
    tasks.extend(download_git(extension_urls,f'{path}/{repo2}/custom_nodes'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
#下载文件2（不要修改）
def download_2():
    tasks = []
    tasks.extend(download_aria('https://developer.download.nvidia.com/compute/cuda/11.7.1/local_installers/cuda_11.7.1_515.65.01_linux.run',f'{path}/{repo2}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo2}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl?download=true',f'{path}/{repo2}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torchaudio-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo2}'))
    tasks.extend(download_aria('https://huggingface.co/Gluttony10/1/resolve/main/whl/torchvision-0.16.2%2Bcu118-cp310-cp310-linux_x86_64.whl?download=true',f'{path}/{repo2}'))
    #tasks.extend(download_aria(embedding_urls,f'{path}/{repo}/models/embeddings'))
    #tasks.extend(download_aria(model_urls,f'{path}/{repo}/models/checkpoints'))
    #tasks.extend(download_aria(unet_urls,f'{path}/{repo}/models/unet'))
    #tasks.extend(download_aria(VAE_urls,f'{path}/{repo}/models/vae'))
    #tasks.extend(download_aria(lora_urls,f'{path}/{repo}/models/loras'))
    #tasks.extend(download_aria(ControlNet_urls,f'{path}/{repo}/models/controlnet'))
    #tasks.extend(download_aria(video_urls,f'{path}/{repo}/input/video'))
    #if os.path.exists(f'{path}/{repo}/custom_nodes/comfyui-animatediff'):
        #tasks.extend(download_aria('https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt?download=true',f'{path}/{repo}/custom_nodes/comfyui-animatediff/models'))
        #tasks.extend(download_aria(motion_lora_urls,f'{path}/{repo}/custom_nodes/comfyui-animatediff/loras'))
    #if os.path.exists(f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved'):
        #tasks.extend(download_aria('https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt?download=true',f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved/models'))
        #tasks.extend(download_aria(motion_lora_urls,f'{path}/{repo}/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora'))
    tasks.extend(download_aria('https://download.pytorch.org/models/mobilenet_v2-b0353104.pth','/root/.cache/torch/hub/checkpoints'))
    tasks.extend(download_aria('https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth','/root/.cache/torch/hub/checkpoints'))
    #tasks.extend(download_hf('CIDAS/clipseg-rd64-refined',f'{path}/{repo}/models/clipseg'))
    #tasks.extend(download_aria('https://github.com/enesmsahin/simple-lama-inpainting/releases/download/v0.1.0/big-lama.pt',f'{path}/{repo}/models/lama'))
    #tasks.extend(download_hf('Salesforce/blip-image-captioning-base',f'{path}/{repo}/models/clip_interrogator/Salesforce/blip-image-captioning-base'))
    #tasks.extend(download_hf('succinctly/text2image-prompt-generator',f'{path}/{repo}/models/prompt_generator/text2image-prompt-generator'))
    #tasks.extend(download_hf('Helsinki-NLP/opus-mt-zh-en',f'{path}/{repo}/models/prompt_generator/opus-mt-zh-en'))
    tasks.extend(download_aria(extra_urls,f'{path}'))
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            executor.submit(os.system, task)
    print('文件下载完毕')
#下载git库（不要修改）
def download_git(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        tasks.append(f'git -C {folder} clone {li} --recursive > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} pull > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} submodule init > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} submodule update > /dev/null 2>&1')
    for li in link:
        name = li.split('/')[-1]
        tasks.append(f'git -C {folder}/{name} fetch > /dev/null 2>&1')
    return tasks
#下载模块（不要修改）
def download_aria(links,folder):
    tasks = []
    link = links.strip().split("\n")
    for li in link:
        if "?download=true" in li:
            li = li.replace("?download=true","")
        if "https://mirror.ghproxy.com/https://github.com" in li:
            li=li.replace("https://mirror.ghproxy.com/https://github.com","https://mirror.ghproxy.com/https://github.com")
        elif "https://github.com" in li:
            li=li.replace("https://github.com","https://mirror.ghproxy.com/https://github.com")
        elif "huggingface.co" in li:
            li=li.replace("huggingface.co","hf-mirror.com")
        elif "civitai.com" in li:
            li=li.replace("civitai.com","civitai.sukaka.top")
        fi = li.split('/')[-1]
        if "." in fi:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} -o {fi} "{li}" > /dev/null 2>&1')
        else:
            tasks.append(f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -t 10 -d {folder} "{li}" > /dev/null 2>&1')
    return tasks
#下载抱脸库（不要修改）

#主进程（不要修改）
def main():
    time_start = time.time()
    #!git config --global url."https://mirror.ghproxy.com/https://github.com".insteadOf "https://github.com"
    #!git config --global url."https://mirror.ghproxy.com/https://raw.githubusercontent.com".insteadOf "https://raw.githubusercontent.com"
    print("运行开始")
    #!df -hl #查看磁盘
    !nvidia-smi #查看显卡
    %cd {path}
    #if reinstall:
       # print('旧文件删除中')
       # !rm -rf {path}/{repo}
    print(f'开始安装{repo2}')
    !git config --global http.postBuffer 2000003072
    #!mkdir -p {path}/{repo}/
    #!git -C {path} clone {url} --recursive > /dev/null 2>&1
    !git -C {path} clone {url2} --recursive > /dev/null 2>&1
    #if os.path.exists(f'{path}/{repo}'):
        #!mkdir -p {path}/{repo}/repositories > /dev/null 2>&1
        #!git -C {path}/{repo} checkout {branch}
        #!git -C {path}/{repo} pull
        #!git -C {path}/{repo} submodule init > /dev/null 2>&1
        #!git -C {path}/{repo} submodule update > /dev/null 2>&1
    if os.path.exists(f'{path}/{repo2}'):
        !mkdir -p {path}/{repo2}/repositories > /dev/null 2>&1
        !git -C {path}/{repo2} checkout {branch2}
        !git -C {path}/{repo2} pull
        !git -C {path}/{repo2} submodule init > /dev/null 2>&1
        !git -C {path}/{repo2} submodule update > /dev/null 2>&1
        print("多进程开始，请耐心等待")
        with ProcessPoolExecutor() as executor:
            futures = []
            for func in [download_1,download_2,install_main,libtcmalloc]:
                futures.append(executor.submit(func))
                time.sleep(10)
            for future in futures:
                future.result()
        %cd {path}/{repo2}
        !pip install torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install torchaudio-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install torchvision-0.16.2%2Bcu118-cp310-cp310-linux_x86_64.whl > /dev/null 2>&1
        !pip install xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl > /dev/null 2>&1
        !sh cuda_11.7.1_515.65.01_linux.run --toolkit --silent
        time_end = time.time()
        print('\033[32m安装耗时:',int((time_end - time_start)/60),'min\033[0m')
        #%cd {path}/{repo}
        #!HF_ENDPOINT=https://hf-mirror.com python main.py --use-pytorch-cross-attention
    else:
        print('安装失败请重试')

check = False
if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


运行开始
Sun Apr 14 14:41:59 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          Off  | 00000000:00:08.0 Off |                    0 |
|  0%   26C    P8     9W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

## 二、安装webui

#### 1. 拉取本体

In [2]:
# 拉取webui本体
!rm -rf /mnt/workspace/stable-diffusion-webui/
%cd  /mnt/workspace/
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"

%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"

/mnt/workspace
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (409/409), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 32887 (delta 248), reused 339 (delta 193), pack-reused 32478
Receiving objects: 100% (32887/32887), 31.66 MiB | 49.58 MiB/s, done.
Resolving deltas: 100% (23700/23700), done.
/mnt/workspace/stable-diffusion-webui
/mnt/workspace/stable-diffusion-webui/repositories
Cloning into 'stable-diffusion-stability-ai'...
remote: Enumerating objects: 574, done.
remote: Counting objects: 100% (574/574), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 574 (delta 266), reused 574 (delta 266), pack-reused 0
Receiving objects: 100% (574/574), 73.48 MiB | 91.09 MiB/s, done.
Resolving deltas: 100% (266/266), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1342, done.
remote: Counting objects: 100% (1342/1342), done.
remote: Compressing objects: 100% (1

#### 2. 安装插件

In [3]:
## 安装常用插件，填写github链接，一行一个
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/sd-webui-infinite-image-browsing
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
"""

extensions = extension_urls.strip().split("\n")

%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

/mnt/workspace/stable-diffusion-webui
Cloning into 'a1111-sd-webui-tagcomplete'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 2281 (delta 151), reused 109 (delta 71), pack-reused 2057
Receiving objects: 100% (2281/2281), 6.72 MiB | 10.94 MiB/s, done.
Resolving deltas: 100% (1459/1459), done.
Cloning into 'stable-diffusion-webui-localization-zh_Hans'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 3237 (delta 131), reused 172 (delta 123), pack-reused 3057
Receiving objects: 100% (3237/3237), 2.06 MiB | 5.51 MiB/s, done.
Resolving deltas: 100% (2140/2140), done.
Cloning into 'sd-webui-additional-networks'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (809/809), done.
remote: Compressing objects: 100% (371/371), done.
remote: Total 809 (delta 457), 

#### 3. 下载模型

In [5]:
#直链下载
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

model_url = '"https://liblibai-online.vibrou.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors?attname=%E7%9C%9F%E5%AE%9E%E6%84%9F%E5%BF%85%E5%A4%87%E6%A8%A1%E5%9E%8B%EF%BD%9CChilloutMix_Chilloutmix-Ni-pruned-fp32-fix.safetensors"'
aria2(model_url, "chilloutmix_NiPrunedFp32Fix.safetensors", "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion")

[#bd9fb2 3.9GiB/3.9GiB(99%) CN:4 DL:165MiB]
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
bd9fb2|OK  |   129MiB/s|/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion/chilloutmix_NiPrunedFp32Fix.safetensors

Status Legend:
(OK):download completed.


In [ ]:
#复制oss内的模型
原始名称 = "models/chilloutmix_Ni.safetensors" #可包含父路径，最前不用加/
标准名称="model.safetensors" #双引号内填模型命名及后缀
!cp /mnt/data/"{原始名称}" /mnt/workspace/lora-scripts/sd-models/"{标准名称}" && echo 复制成功

#### 4. 下载 VAE

In [6]:
VAE_URL = '"https://modelscope.cn/api/v1/models/dienstag/anything-v4.0/repo?Revision=master&FilePath=anything-v4.0.vae.pt"'
aria2(VAE_URL, "animevae.pt", "/mnt/workspace/stable-diffusion-webui/models/VAE")

[#680891 658MiB/784MiB(83%) CN:13 DL:445MiB]]
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
680891|OK  |   432MiB/s|/mnt/workspace/stable-diffusion-webui/models/VAE/animevae.pt

Status Legend:
(OK):download completed.


#### 5. 下载文件（样例）

In [ ]:
# 这部分是可选的！！！！！有需要再执行！！！！！！！ 修改对应文本，运行即可下载文件
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

DOWNLOAD_URL = "文件链接" # 文件链接，需要下载的url直链才可以
FILE_NAME = "aki" # 下载保存的文件名
SAVE_DIR = "/mnt/workspace" # 文件保存的路径
aria2(DOWNLOAD_URL, FILE_NAME, SAVE_DIR)

## 三、 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
#cuda_path = "/usr/local/cuda-11.7/targets/x86_64-linux/lib"
#ld_library_path = os.environ.get("LD_LIBRARY_PATH")
#ld_library_path = f"{ld_library_path}:{cuda_path}"
#os.environ["LD_LIBRARY_PATH"] = ld_library_path
os.environ["PIP_INDEX_URL"] = "https://mirror.baidu.com/pypi/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access

/mnt/workspace/stable-diffusion-webui
Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Version: v1.9.0
Commit hash: adadb4e3c7382bf3e4f7519126cd6c70f4f8557b
Cloning Stable Diffusion XL into /mnt/workspace/stable-diffusion-webui/repositories/generative-models...
Cloning into '/mnt/workspace/stable-diffusion-webui/repositories/generative-models'...
remote: Enumerating objects: 941, done.


### 四、 启动Lora
- 停止后再顺序运行

In [ ]:
#每次启动实例只运行一次 等待出现gradio网址后，再运行下一步
import threading
def start_gradio():
    !gradio-tunneling --port 28000
background_thread = threading.Thread(target=start_gradio)
background_thread.daemon = True
background_thread.start()

In [ ]:
import os
cuda_path = "/usr/local/cuda-11.7/targets/x86_64-linux/lib"
ld_library_path = os.environ.get("LD_LIBRARY_PATH")
ld_library_path = f"{ld_library_path}:{cuda_path}"
os.environ["LD_LIBRARY_PATH"] = ld_library_path
%cd /mnt/workspace/lora-scripts
!pip uninstall tb-nightly tensorboard tensorflow tensorflow-estimator tf-estimator-nightly tf-nightly -y > /dev/null 2>&1
!pip install -r requirements.txt > /dev/null 2>&1
!HF_ENDPOINT=https://hf-mirror.com python gui.py